# Projet statistique descriptive 



In [2]:
%reload_ext autoreload
%autoreload 2

# module
import sys
sys.path.append("../")
from src import * 

# chemin vers la donnee 
data_dir = 'C:/Users/felix/PycharmProjects/pythonProject1/MACS3-Statistiques-Descriptives-TDs/data/archive/'
filelist = glob.glob(data_dir + '*.h5')
filelist

IndentationError: unindent does not match any outer indentation level (manipulate_dataframe.py, line 236)

In [ ]:
from tabata import Opset
#première visualisation brut de la données 

# Première visualisation brut de la données 
La visualisation des graphes nous à permis dans un premier temps de constater des anomalies dans les données et de supprimer ces données pour la suite de nos analyse.En effet ces données dites anormales représentent des mesures ou l'avions n'à pas décollé (sûrement des tests au sols) donc il est légitimes de les supprimer quand on s'intéresse aux comportements de l'avion en vol.

In [ ]:
df_opset1=Opset('C:/Users/felix/PycharmProjects/pythonProject1/MACS3-Statistiques-Descriptives-TDs/data/archive\\Aircraft_01.h5')
df_opset2=Opset('C:/Users/felix/PycharmProjects/pythonProject1/MACS3-Statistiques-Descriptives-TDs/data/archive\\Aircraft_02.h5')
df_opset3=Opset('C:/Users/felix/PycharmProjects/pythonProject1/MACS3-Statistiques-Descriptives-TDs/data/archive\\Aircraft_03.h5')

In [ ]:
df_opset1

In [ ]:
df_opset1.plot()

In [ ]:
type(df_opset2)

In [ ]:
df_opset2.plot()

In [ ]:
df_opset3.plot()

## Creation d'une fonction sur les dataframes pour éliminer les records anormaux
On utilise l'avantage de Dask qui permet de paralléliser sur les dataframes.
Le but de la fonction eliminate_records() est de vérifier la différence entre l'altitude minimum et maximum est d'au plus 15000 ft si c'est le cas alors elle renvoit False et True sinon.



In [ ]:
def eliminate_records(df):
    
    alt_min=df['ALT [ft]'].min()
    alt_max=df['ALT [ft]'].max()
    alt_range=alt_max-alt_min
    bol=False
    
    if alt_range<15000:
        bol=True
    return bol
    
    

In [ ]:
#on cree notre dataframe avec DASK
ddf1 = lire_hdf_dask(filelist[0], repertoire= data_dir)
#on récupère une liste de booleen où chaque composante fait reférance à une partition 
I = ddf1.map_partitions(lambda df: eliminate_records(df)).compute()
#on sélectione dans un nouveau dataframe les partitions qui ne sont pas anormales 
DDF1=ddf1.partitions[I==False]
# on normalise toute les données excepté les variables booleen 
ddf_norm1 = DDF1.map_partitions(lambda df : normalize(df))

ddf2 = lire_hdf_dask(filelist[2], repertoire= data_dir)
I = ddf2.map_partitions(lambda df: eliminate_records(df)).compute()
DDF2=ddf2.partitions[I==False]
ddf_norm2 = DDF2.map_partitions(lambda df : normalize(df))

ddf3 = lire_hdf_dask(filelist[3], repertoire= data_dir)
I = ddf3.map_partitions(lambda df: eliminate_records(df)).compute()
DDF3=ddf3.partitions[I==False]
ddf_norm3 = DDF3.map_partitions(lambda df : normalize(df))

In [ ]:
ddf_norm2.head()

# Etude de la relation entre monte et EGT 
-On cherche à trouver des relations entre EGT max et la vitesse de monte par exemple 



Il va falloir peut être modifier la fonction deriv_glissante lorsque qu'on passera à l'analyse de chaque record 

In [ ]:
def deriv_glissante(df):
    
    #on cree une fonction qui prend un data frame qui l'augment de la valeur dy et qui renvoit  un nouveau dataframe contenant que le monté 
    #ATTENTION LA FONCTION PRENDS EN ENTREE DES DONNEES NORMALISEE
    
    dy=np.zeros(len(df))
    
    #on fait une moyenne glissante pour éviter d'être sensible aux bruit. En effet pour le cacul de dérivée le bruit 
    #fausse trés rapidement les données
    y = df['ALT'].rolling(window = 20).mean()
    I = range(y.index[0], y.index[-1],20)
    #calcul de la derivée
    dy[0:len(I)-1] = y[I[1:]].values- y[I[:-1]].values
    #creation d'une nouvelle colonne dans le dataframe
    df['dy']=dy
    #on selectionne le dataframe où les dérivées sont plus grande que 0.01 (la montée)
    df_monte=df[df['dy']>0.01]
    I4=np.zeros(len(df_monte))
    #on verifie qu' avec un shift des données on retombe bien sur les mêmes indices cela permet d'éviter 
    # de prendre en compte des valeurs qui ne serait pas dans la phase de la montée 
    #Attention cette façon de faire est lié aux cas d'un vol d'avion (l'hypothèse admise est que l'avion ne monte qu'une fois)
    #Et s'il existe plus de 6 données anormale consécutives alors elles seront prise en compte 
    
    I4[5:]=df_monte.index[5:]==(df_monte.index+5)[:-5]
    I4[:5]=1
    df_monte2=df_monte[I4==1]
    


    return df_monte2

In [ ]:
_VOL=10
DF_monte=ddf_norm1.map_partitions(lambda df:deriv_glissante(df))
DF_monte.partitions[_VOL].compute()

In [ ]:
df=ddf_norm1.partitions[10].compute()
y = df['ALT'].rolling(window = 20).mean()
I = range(y.index[0], y.index[-1],20)
dy = y[I[1:]].values- y[I[:-1]].values
plt.plot(I[1:], dy)
len(y)

In [ ]:
df12=ddf_norm1.partitions[100].compute()

df_monte2=deriv_glissante(df12)


#plt.plot(df12['EGT_1'][I_monte])


#duree de la monte
duree=len(df_monte2)*20
#Indice monte
I_monte=df_monte2.index*20
#calcul de l'egt max pour chaque moteur pendant la monté
EGT_max_monte1=df['EGT_1'][I_monte].max()
EGT_max_monte2=df['EGT_2'][I_monte].max()
EGT_max_monte=max(EGT_max_monte1,EGT_max_monte2)
df12['dy'].max()

peut paralléliser avec dask 

In [ ]:
duree11=[]
EGT_max_monte1=[]
EGT_mean_monte1=[]
Vitesse_max=[]
I=[]
for i in range (0,990):
    I.append(i)
    df11=ddf_norm1.partitions[i].compute()
    #df22=ddf_norm2.partitions[i].compute()
    #df22=ddf_norm3.partitions[i].compute()
    df_monte11=deriv_glissante(df11)
    #[df_monte22]=deriv_glissante(df22)
    #[df_monte33]=deriv_glissante(df33)
    
    duree11.append(len(df_monte11)*20)
    #duree22=len(df_monte22)*20
    #duree33=len(df_monte33)*20

    #Indice monte
    I_monte11=df_monte11.index*20
    #I_monte22=df_monte22.index*20
    #I_monte33=df_monte33.index*20
    #calcul de la vittesse max pendant la monte 
    Vitesse_max.append(df_monte11['dy'].mean())
    #calcul de l'egt max pour chaque moteur pendant la monté
    
    EGT_max_monte11=df11['EGT_1'][I_monte11].max()
    EGT_max_monte21=df11['EGT_2'][I_monte11].max()
    
    #EGT_mean_monte11=df['EGT_1'][I_monte11].mean()
    #EGT_mean_monte21=df['EGT_2'][I_monte11].mean()   
    #EGT_max_monte12=df['EGT_1'][I_monte22].max()
    #EGT_max_monte22=df['EGT_2'][I_monte22].max()
    
    #EGT_max_monte13=df['EGT_1'][I_monte33].max()
    #EGT_max_monte23=df['EGT_2'][I_monte33].max()
    #EGT_mean_monte1.append(EGT_max_monte11)
    EGT_max_monte1.append(max(EGT_max_monte11,EGT_max_monte21))
    #EGT_max_monte2.append(max(EGT_max_monte12,EGT_max_monte22))    
    #EGT_max_monte3.append(max(EGT_max_monte13,EGT_max_monte23))    


In [ ]:
egtm = EGT_max_monte1.rolling(window=15).median()
plt.figure(figsize=(10,8))
plt.plot(EGT_max_monte1.index, egt, 'rx')
plt.plot(egtm.index, egtm, '-', c='purple')
plt.legend(['Marge EGT', 'Marge EGT (médiane sur 15 cycles)'])
plt.title('Marge EGT et médiane glissante sur 15 cycles')

In [ ]:
Vitesse_max

In [ ]:
plt.scatter(Vitesse_max,EGT_max_monte1)

In [ ]:
plt.scatter(duree11,EGT_mean_monte1)

In [ ]:
plt.scatter(duree11,EGT_max_monte1)

In [ ]:
EGT_max_monte1

In [ ]:
from sklearn.cluster import KMeans

In [ ]:
Array2D[:,1]

In [ ]:
K = 3
Array2D=np.column_stack((EGT_max_monte1,duree11))
km = KMeans(n_clusters=K, n_init=10)
labels = km.fit_predict(Array2D)
cnt = np.bincount(labels)

fig,ax = plt.subplots( 2, figsize=(15,10))

ax[0].bar(np.arange(K), cnt, tick_label=np.arange(K))
ax[0].set_title('Nombre d\'éléments par cluster')
ax[0].set_xlabel('Cluster')
for c in range(km.n_clusters):
    ax[1].scatter(Array2D[labels == c, 1], Array2D[labels == c, 0], label="Cluster "+str(c))
ax[1].legend()
ax[1].set_xlabel('duree')
ax[1].set_ylabel('EGT max')

In [ ]:
plt.scatter(I,EGT_max_monte1)

# Suite de l'analyse Corelation et ACP en montee

In [ ]:
# peut on prédire l'egt en fonction des température et des autre valeurs 

In [ ]:
#peut on analyser les données manquantes et peut être apporter un corectif  